In [1]:
import pandas as pd
import re

In [2]:
s_loc = './data/sample/'
d_sets = ['Black-Male-Names.csv','Black-Female-Names.csv','Hispanic-Male-Names.csv','Hispanic-Female-Names.csv','White-Male-Names.csv','White-Female-Names.csv','Indian-Male-Names.csv','Indian-Female-Names.csv']
d_sets

['Black-Male-Names.csv',
 'Black-Female-Names.csv',
 'Hispanic-Male-Names.csv',
 'Hispanic-Female-Names.csv',
 'White-Male-Names.csv',
 'White-Female-Names.csv',
 'Indian-Male-Names.csv',
 'Indian-Female-Names.csv']

In [3]:
map_info = {'gender': {0:'Male',1:'Female'}, 'race' : {0:'black',1:'hispanic',2:'white',3:'indian'}}
map_info

{'gender': {0: 'Male', 1: 'Female'},
 'race': {0: 'black', 1: 'hispanic', 2: 'white', 3: 'indian'}}

In [4]:
data = pd.DataFrame([])

In [6]:
rows_c = 165
for i in d_sets:
    temp = pd.read_csv(s_loc+i,nrows=rows_c)
    data = data.append(temp)

In [7]:
data.head()

,name,gender,race,length
0,aaric,0,0,5
1,dominique,0,0,9
2,fredrick,0,0,8
3,jarvis,0,0,6
4,lorenzo,0,0,7


In [11]:
data.gender.value_counts()

1    660
0    660
Name: gender, dtype: int64

In [12]:
data.race.value_counts()

3    330
2    330
1    330
0    330
Name: race, dtype: int64

#### X & Y

In [17]:
X = data.name.values

In [19]:
y = data.race.values
len(y)

1320

### Train-Test split

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [23]:
pd.Series(y_test).value_counts()

1    78
0    63
2    62
3    61
dtype: int64

### Utiities

In [34]:
def name_prep(word):
    #te = re.sub(r'/.*|@.*','',word).strip()
    te = str(word).lower()
    te = re.sub(r'[^a-z.\s].*','',te).strip()
    te = re.sub(r'[^a-z ]+','',te).strip()
    te = re.sub(r' +',' ',te).strip()
    te = re.sub(r'along.*|with.*|and.*',' ',te).strip()
    #te = re.sub(r'\b\w$|\b\w\w$','',te).strip()
    te = re.sub(r'\b\w\b|\b\w\w\b','',te).strip()
    #te = re.sub(r'\bna\b|\bNaN\b','',te).strip()
    if(len(te) < 3):
        return None
    else:
        return te

In [51]:
def conv_input(name,vectorizer):
    pre = name
    return vectorizer.transform([pre])

In [50]:
def predict_from_name(model,name,vectorizer):
    pre = name_prep(name)
    if(pre is None):
        print("Invalid !!!")
        return
    else:
        p = model.predict(conv_input(pre,vectorizer).toarray())
        print("Race :",map_info['race'][p[0]])

### Vectorization

In [24]:
def name_to_list(names):
    return list(map(lambda x: str(x),names))

In [25]:
names_list = name_to_list(X_train)

* Character count Vectorizer

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
char_vec = CountVectorizer(lowercase=True,token_pattern='.')

In [29]:
char_name_vec = char_vec.fit_transform(names_list)

In [30]:
char_name_vec

<1056x27 sparse matrix of type '<class 'numpy.int64'>'
	with 6056 stored elements in Compressed Sparse Row format>

In [32]:
print(char_vec.vocabulary_)

{'a': 1, 'n': 14, 'g': 7, 'e': 5, 'l': 12, 'k': 11, 'j': 10, ' ': 0, 'c': 3, 'h': 8, 'o': 15, 'u': 21, 'd': 4, 'r': 18, 'y': 25, 'i': 9, 'b': 2, 't': 20, 'm': 13, 'v': 22, 's': 19, 'x': 24, 'w': 23, 'f': 6, 'p': 16, 'z': 26, 'q': 17}


### Model

In [36]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [37]:
from sklearn.naive_bayes import GaussianNB

* Naive Bayes

In [38]:
model_1 = 'NavieB'

In [39]:
naive_b = GaussianNB()

In [40]:
naive_b.fit(char_name_vec.toarray(),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [52]:
p = naive_b.predict(conv_input('Narendra Modi',char_vec).toarray())

In [58]:
predict_from_name(naive_b,'Narendra Modi',char_vec)

Race : indian
